## Load & Preprocess the Dataset
We will use the IMDB dataset from TensorFlow/Keras instead of NLTK for compatibility with deep learning models.

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load dataset (top 10,000 most frequent words)
num_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

print(f"Training Samples: {len(X_train)}, Testing Samples: {len(X_test)}")
print(f"Example review (encoded): {X_train[0]}")
print(f"Label: {y_train[0]} (0 = Negative, 1 = Positive)")


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training Samples: 25000, Testing Samples: 25000
Example review (encoded): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12

## Pad Sequences for Uniform Length
LSTMs require fixed-length inputs, so we pad shorter reviews and truncate longer ones.

In [2]:
max_length = 200  # Maximum number of words per review
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

print(f"Post-padding shape: {X_train.shape}")

Post-padding shape: (25000, 200)


## Build the LSTM Model
✅ Embedding layer (for word embeddings)

✅ LSTM layer (to capture sequential patterns)

✅ Dense layers (for classification)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define model
model = Sequential([
    Embedding(input_dim=num_words, output_dim=128, input_length=max_length),  # Word Embeddings
    LSTM(128, return_sequences=True),  # LSTM Layer 1
    LSTM(64),  # LSTM Layer 2
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Output layer (Binary classification)
])

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Train The Model

In [4]:
# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 435s 1s/step - accuracy: 0.5104 - loss: 0.6933 - val_accuracy: 0.5544 - val_loss: 0.6833
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 437s 1s/step - accuracy: 0.5918 - loss: 0.6587 - val_accuracy: 0.6221 - val_loss: 0.6497
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 397s 971ms/step - accuracy: 0.7439 - loss: 0.5464 - val_accuracy: 0.7968 - val_loss: 0.4960
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 431s 1s/step - accuracy: 0.8598 - loss: 0.3644 - val_accuracy: 0.8432 - val_loss: 0.3777
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 441s 1s/step - accuracy: 0.8984 - loss: 0.2820 - val_accuracy: 0.8242 - val_loss: 0.3890


## Evaluate the model

In [5]:
# Evaluate performance
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 120s 153ms/step - accuracy: 0.8220 - loss: 0.3933
Test Accuracy: 0.82


## Predict Sentiment for Custom Reviews
We will write a function to convert text to model input and make predictions

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Load Tokenizer (same vocab as IMDB)
tokenizer = Tokenizer(num_words=num_words)

def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])  # Convert to sequence
    padded = pad_sequences(sequence, maxlen=max_length)  # Pad sequence
    prediction = model.predict(padded)[0][0]  # Get prediction
    return "Positive" if prediction > 0.5 else "Negative"

# Test the function
print(predict_sentiment("This movie was absolutely fantastic!"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Positive
